In [ ]:
import sys
import torch
import matplotlib.pyplot as plt
from utils import Parameters
from model_wrapper import ModelWrapper
from dataloaders import LightningWrapperData


In [ ]:
config_file =  "../configs/test.yaml"

#if len(sys.argv) > 1:
#    params = Parameters()
#    config_file = sys.argv[1]
#    params.GetParams(config_file)
#else:
#    print("*.yaml input file needed as command line argument")
#    exit()

params = Parameters()
#config_file = sys.argv[1]
params.GetParams(config_file)
print(params.n_classes) #.keys())

model = ModelWrapper(params)
model.load(params.load_checkpoint)

dataset = LightningWrapperData(params)
dataset.setup("train")
testing = dataset.test_dataloader()

predictions = []
inputs = []
targets = []
contador = 0
with torch.no_grad():
    for batch in testing:
        input_ = batch["image"]
        target = batch[params.target_label]
        pred = model.forward(input_)
#            print(" SHAPES DEL BATCH")
#            print("inputs",input.shape)
#            print("target",target.shape)
        print("pred", pred.shape, pred.max())
        inputs.append(input_)
        targets.append(target)
        predictions.append(pred)
        contador = contador +1
        if contador > 10:
            break
    
_pred = torch.cat(predictions, dim=0)
all_pred = torch.argmax(_pred,dim=1)
all_in = torch.cat(inputs,dim=0)
all_tar = torch.cat(targets, dim=0)
data = {"predictions":all_pred, "all_inputs":all_in, "all_targets":all_tar}
# Temporalmente queda algo cutre pero después agrégale una os.path join y tal
# Alternativamente podríamos agregar otra variable que tenga libertad de elegir
# el nombre pero ya me parece un poco innecesario para usar con params.log_path
torch.save(data, "prediction.pt")


In [ ]:
predictions = torch.load("prediction.pt")

In [ ]:
for muestra in range(len(predictions["all_inputs"])):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    im1 = axs[0].imshow(predictions["all_inputs"][muestra,0,:,:].detach().numpy(), cmap='viridis', aspect='auto')
    axs[0].set_title('Input', fontsize=25)

    im2 = axs[1].imshow(predictions["all_targets"][muestra,0,:,:].detach().numpy(), cmap='inferno', aspect='auto')
    axs[1].set_title('Target', fontsize=25)

    im3 = axs[2].imshow(predictions["predictions"][muestra,:,:].detach().numpy(), cmap='inferno', aspect='auto')

    axs[2].set_title('Prediction', fontsize=25)

    plt.tight_layout()

    plt.show()